In [5]:
"""
Creates catalogues of JADES observations, split into spectral, photo, and overlap catalogues.

Utilises released catalogues from https://archive.stsci.edu/hlsp/jades.

NIRSpec:
    DR3 (v1.1 files) contain DR1 (v1.0) observations so only the DR3 catalogue per field is needed.
    Columns described in https://archive.stsci.edu/hlsps/jades/hlsp_jades_jwst_nircam-nirspec_goods-ns-all_multi_dr3_readme.txt
    Individual spectra have to be downloaded separately, emission line data is within catalogue, but not continuum data.

NIRCam:
    Subsequent photometry has superceded previous versions, one catalogue per field.
    Columns described in https://archive.stsci.edu/hlsps/jades/hlsp_jades_jwst_nircam_goods-s-deep_photometry_v2.0_catalog-ext-readme.pdf
    JEMS and FRESCO observations are included, leads to inconsistent filters for certain observations, missing filters have 0 or NaN flux.

The JADES catalogues have been renamed '{instrument} {data-release} {field} catalogue.fits' i.e. 'NIRCam DR2 GS catalogue.fits'
"""

In [1]:
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import pandas as pd

from msc_funcs import *

# Spectral Catalogue:

Creates .csv catalogue of all high-z spectra with reliably measured betas.

Some betas cannot be fitted. They are given the following codes depending on the error:
 - 1111 (FileNotFoundError): 1D spectra file does not exist;
 - 2222 (RuntimeError): Unable to fit beta (least-squares failure, see curve_fit);
 - 3333: Differently scaled spectra give different (non-agreeing) values of beta. See spectral_fit;
 - 4444 (ValueError): Only for photometry, see curve_fit (NaNs? shouldn't have any but maybe?);
 - 5555: Only for photometry, when fewer than two filters lie within range.


In [ ]:
# Lists which will be populated from catalogue data.
# In DR3 {TIER}-{NIRSPEC_ID} combination is the unique identifier, not NIRSPEC_ID alone.
nirspec_ids = []
nircam_ids = []
nirspec_zs = []
nirspec_ra = []
nirspec_dec = []
nirspec_exp = []
DR_flag = []


# For each catalogue file collect high-z spectra data
for field in ["GN","GS"]:
    hdul = fits.open(f"JADES/NIRSpec DR3 {field} catalogue.fits")  # custom name and 
    t = Table(hdul[1].data).to_pandas().query("z_Spec >= 5").reset_index(drop = True)
    display(t)
    for i in range(len(t)):
        nirspec_ids.append(f"{t.TIER[i]}-{t.NIRSpec_ID[i]:08}")
        nircam_ids.append(t.NIRCam_ID[i])
        nirspec_zs.append(t.z_Spec[i])
        nirspec_ra.append(t.RA_TARG[i])
        nirspec_dec.append(t.Dec_TARG[i])
        nirspec_exp.append(t.tExp_Prism[i])
        DR_flag.append(t.DR_flag[i])

# Standardise NIRCam_ID of those spectra with no NIRCam observations
nircam_ids = [id if id != 999999 and id > 0 else None for id in nircam_ids]

In [ ]:
# Fits beta for each NIRSpec observation.

# Lists
beta1300_2500  , beta1300_3000  , beta1300_3500   = [], [], []
beta1500_2500  , beta1500_3000  , beta1500_3500   = [], [], []

beta1300_2500_e, beta1300_3000_e, beta1300_3500_e = [], [], []
beta1500_2500_e, beta1500_3000_e, beta1500_3500_e = [], [], []

betacalzetti, betacalzetti_e = [], []

# Fitting
for id, z in zip(nirspec_ids, nirspec_zs, strict = True):
    try:
        hdul = fits.open(f"JADES/All Spectra/hlsp_jades_jwst_nirspec_{id}_clear-prism_v1.0_x1d.fits")

        #drop NaN to allow fitting even with partial/incomplete spectra 
        t = Table(hdul[1].data).to_pandas().dropna(subset = ["WAVELENGTH", "FLUX"])

        lam = t.WAVELENGTH.to_numpy()*1e4  # originally in um, now AA
        flam = t.FLUX.to_numpy()  # erg/s/cm**2/AA
        flam_e = t.FLUX_ERR.to_numpy()# erg/s/cm**2/AA

        # 1300 - 2500
        beta, betaerror = spectral_fit(1300., 2500., z, lam, flam, flam_e)
        beta1300_2500.append(beta), beta1300_2500_e.append(betaerror)
        # 1300 - 3000
        beta, betaerror = spectral_fit(1300., 3000., z, lam, flam, flam_e)
        beta1300_3000.append(beta), beta1300_3000_e.append(betaerror)
        # 1300 - 3500
        beta, betaerror = spectral_fit(1300., 3500., z, lam, flam, flam_e)
        beta1300_3500.append(beta), beta1300_3500_e.append(betaerror)

        # 1500 - 2500
        beta, betaerror = spectral_fit(1500., 2500., z, lam, flam, flam_e)
        beta1500_2500.append(beta), beta1500_2500_e.append(betaerror)
        # 1500 - 3000
        beta, betaerror = spectral_fit(1500., 3000., z, lam, flam, flam_e)
        beta1500_3000.append(beta), beta1500_3000_e.append(betaerror)
        # 1500 - 3500
        beta, betaerror = spectral_fit(1500., 3500., z, lam, flam, flam_e)
        beta1500_3500.append(beta), beta1500_3500_e.append(betaerror)

        # Spectral Calzetti
        beta, betaerror = spectral_calzetti(z, lam, flam, flam_e)
        betacalzetti.append(beta), betacalzetti_e.append(betaerror)
    except FileNotFoundError:
        # There are some observations which exist in the catalogue but have no PRISM
        # spectra in MAST. Potentially only observed at higher resolutions?
        beta1300_2500.append(1111), beta1300_2500_e.append(1111)
        beta1300_3000.append(1111), beta1300_3000_e.append(1111)
        beta1300_3500.append(1111), beta1300_3500_e.append(1111)
        beta1500_2500.append(1111), beta1500_2500_e.append(1111)
        beta1500_3000.append(1111), beta1500_3000_e.append(1111)
        beta1500_3500.append(1111), beta1500_3500_e.append(1111)
        betacalzetti.append(1111), betacalzetti_e.append(1111)

In [ ]:
# Creates csv to store locally/not have to rerun code.
# Includes all spectra at z>=5. Gives NIRCam ID if observed, None otherwise.
# 3 spectra do not exist, 8 are not reliably fitted.

spectral_catalogue = pd.DataFrame(
        data = zip(nirspec_ids, nircam_ids, nirspec_zs, nirspec_ra, nirspec_dec, nirspec_exp, DR_flag,
                   beta1300_2500, beta1300_3000, beta1300_3500, beta1500_2500, beta1500_3000,
                   beta1500_3500, betacalzetti, beta1300_2500_e, beta1300_3000_e,beta1300_3500_e,
                   beta1500_2500_e, beta1500_3000_e, beta1500_3500_e, betacalzetti_e, strict=True),
        columns = ["NIRSpec_ID", "NIRCam_ID", "z_Spec", "RA_TARG", "Dec_TARG", "tExp", "DR_flag",
                   "beta1300_2500", "beta1300_3000","beta1300_3500", "beta1500_2500", "beta1500_3000",
                   "beta1500_3500", "betacalzetti", "beta1300_2500_e", "beta1300_3000_e",
                   "beta1300_3500_e", "beta1500_2500_e", "beta1500_3000_e", "beta1500_3500_e", "betacalzetti_e"]
        )

# Manually deletes 11 spectra:

# 3 FILENOTFOUND: (potentially not looked by prism but haven't looked into it)
# goods-n-mediumhst-00001203
# goods-n-mediumhst-00004108
# goods-n-mediumhst-00046903

# 8 RUNTIME/UNRELIABLE: (if not able to be fitted over various wavelength ranges it is deleted)
# goods-n-mediumjwst-00013823
# goods-n-mediumhst-00040307
# goods-s-mediumhst-00009768
# goods-s-mediumjwst-00051925
# goods-s-mediumjwst1180-00051925
# goods-s-ultradeep-00101167
# goods-s-ultradeep-00106197
# goods-s-mediumjwst1180-10011541

badspecids= ["goods-n-mediumhst-00001203", "goods-n-mediumhst-00004108", "goods-n-mediumhst-00046903", "goods-n-mediumjwst-00013823", "goods-n-mediumhst-00040307", "goods-s-mediumhst-00009768", "goods-s-mediumjwst-00051925", "goods-s-mediumjwst1180-00051925", "goods-s-ultradeep-00101167", "goods-s-ultradeep-00106197", "goods-s-mediumjwst1180-10011541"]

spectral_catalogue.query("NIRSpec_ID not in @badspecids", inplace = True)
spectral_catalogue.to_csv("spectral catalogue.csv", index = False)

# Photometric Catalogue:

All photometric data is within the JADES catalogues, no need to download other files.

Creates .csv catalogue of all photometric observations.

Combines the GS/GN catalogues into a unified file with both fluxes and redshifts.

<!-- NEED TO MAKE BETA MEASUREMENTS ON THIS DIRECTLY PROBABLY, ALSO HERE HAVE THE HOW MANY FILTERS OBSERVED THING -->


In [ ]:
# Create merged catalogue

# DR2
photo_hdul = fits.open("JADES/NIRCam DR2 GS catalogue.fits")
photo_fnus_df = Table(photo_hdul[8].data).to_pandas()
photo_fnus_df = photo_fnus_df[
    ['ID', 'RA', 'DEC',
     'F090W_KRON', 'F115W_KRON', 'F150W_KRON', 'F182M_KRON', 'F200W_KRON', 'F210M_KRON', 'F277W_KRON',
     'F335M_KRON', 'F356W_KRON', 'F410M_KRON', 'F430M_KRON', 'F444W_KRON', 'F460M_KRON', 'F480M_KRON',
     'F090W_KRON_ei', 'F115W_KRON_ei', 'F150W_KRON_ei', 'F182M_KRON_ei', 'F200W_KRON_ei', 'F210M_KRON_ei', 'F277W_KRON_ei',
     'F335M_KRON_ei', 'F356W_KRON_ei', 'F410M_KRON_ei', 'F430M_KRON_ei', 'F444W_KRON_ei', 'F460M_KRON_ei', 'F480M_KRON_ei']]
photo_zs_df = Table(photo_hdul[9].data).to_pandas()
photo_zs_df = photo_zs_df[['ID', 'EAZY_z_a', 'EAZY_l68', 'EAZY_u68']]
photo_dr2 = photo_fnus_df.merge(photo_zs_df, on = "ID")

# DR3 - different filters (no/some JEMS)
photo_hdul = fits.open("JADES/NIRCam DR3 GN catalogue.fits")
photo_fnus_df = Table(photo_hdul[8].data).to_pandas()
photo_fnus_df = photo_fnus_df[
    ['ID', 'RA', 'DEC',
     'F090W_KRON', 'F115W_KRON', 'F150W_KRON', 'F182M_KRON', 'F200W_KRON', 'F210M_KRON', 'F277W_KRON',
     'F335M_KRON', 'F356W_KRON', 'F410M_KRON',               'F444W_KRON',
     'F090W_KRON_ei', 'F115W_KRON_ei', 'F150W_KRON_ei', 'F182M_KRON_ei', 'F200W_KRON_ei', 'F210M_KRON_ei', 'F277W_KRON_ei',
     'F335M_KRON_ei', 'F356W_KRON_ei', 'F410M_KRON_ei',                 'F444W_KRON_ei'                                ]]
photo_zs_df = Table(photo_hdul[9].data).to_pandas()
photo_zs_df = photo_zs_df[['ID', 'EAZY_z_a', 'EAZY_l68', 'EAZY_u68']]
photo_dr3 = photo_fnus_df.merge(photo_zs_df, on = "ID")

photo_catalogue = pd.concat((photo_dr2, photo_dr3), ignore_index=True).rename(columns={"ID": "NIRCam_ID"})
photo_catalogue.to_csv("photo catalogue.csv", index = False)

# Overlap Catalogue:

Those sources with both NIRSpec and NIRCam are referred to throughout as 'overlap'.

Creates a .csv catalogue with betas (1300-3500 vs photometric) and redshifts (EAZY vs spec).

There are several photo sources with multiple spectroscopic observations. The spectra with the highest exposure time is chosen as the more reliable one.

There are several NIRCam IDs which do not exist in the photometric catalogues, confusing, discussed in thesis.

In [ ]:
# Empty lists
spectral_id = []
spectral_z = []
spectral_beta = []
spectral_error = []
spectral_ra = []
spectral_dec = []
spectral_DR = []

# Import catalogues
spec_cat = pd.read_csv("spectral catalogue.csv", usecols = ["NIRSpec_ID", "NIRCam_ID", "z_Spec", "beta1300_3500", "beta1300_3500_e", "tExp", "RA_TARG", "Dec_TARG", "DR_flag"]).query("beta1300_3500 < 1000").reset_index(drop=True)
photo_cat = pd.read_csv("photo catalogue.csv")

# Get list of all NIRCam IDs linked to spectral observations
spec_nircam = spec_cat.NIRCam_ID.dropna().astype('int64').tolist()

# Disregard IDs which do not exist in the photometric catalogue
photo_overlap = photo_cat.query("NIRCam_ID in @spec_nircam")

# Get data for each NIRCam ID
for nircamid in photo_overlap.NIRCam_ID:
    df = spec_cat.query("NIRCam_ID == @nircamid").reset_index(drop=True)
    if len(df) == 2:
        if df.tExp[0] >= df.tExp[1]:
            spectral_id.append(df.NIRSpec_ID[0])
            spectral_z.append(df.z_Spec[0])
            spectral_beta.append(df.beta1300_3500[0])
            spectral_error.append(df.beta1300_3500_e[0])
            spectral_ra.append(df.RA_TARG[0])
            spectral_dec.append(df.Dec_TARG[0])
            spectral_DR.append(df.DR_flag[0])
        else:
            spectral_id.append(df.NIRSpec_ID[1])
            spectral_z.append(df.z_Spec[1])
            spectral_beta.append(df.beta1300_3500[1])
            spectral_error.append(df.beta1300_3500_e[1])
            spectral_ra.append(df.RA_TARG[1])
            spectral_dec.append(df.Dec_TARG[1])
            spectral_DR.append(df.DR_flag[1])
    elif len(df) == 3:
        if df.tExp[0] >= df.tExp[1] and df.tExp[0] >= df.tExp[2]:
            spectral_id.append(df.NIRSpec_ID[0])
            spectral_z.append(df.z_Spec[0])
            spectral_beta.append(df.beta1300_3500[0])
            spectral_error.append(df.beta1300_3500_e[0])
            spectral_ra.append(df.RA_TARG[0])
            spectral_dec.append(df.Dec_TARG[0])
            spectral_DR.append(df.DR_flag[0])
        elif df.tExp[1] >= df.tExp[0] and df.tExp[1] >= df.tExp[2]:
            spectral_id.append(df.NIRSpec_ID[1])
            spectral_z.append(df.z_Spec[1])
            spectral_beta.append(df.beta1300_3500[1])
            spectral_error.append(df.beta1300_3500_e[1])
            spectral_ra.append(df.RA_TARG[1])
            spectral_dec.append(df.Dec_TARG[1])
            spectral_DR.append(df.DR_flag[1])
        else:
            spectral_id.append(df.NIRSpec_ID[2])
            spectral_z.append(df.z_Spec[2])
            spectral_beta.append(df.beta1300_3500[2])
            spectral_error.append(df.beta1300_3500_e[2])
            spectral_ra.append(df.RA_TARG[2])
            spectral_dec.append(df.Dec_TARG[2])
            spectral_DR.append(df.DR_flag[2])
    else:
        spectral_id.append(df.NIRSpec_ID[0])
        spectral_z.append(df.z_Spec[0])
        spectral_beta.append(df.beta1300_3500[0])
        spectral_error.append(df.beta1300_3500_e[0])
        spectral_ra.append(df.RA_TARG[0])
        spectral_dec.append(df.Dec_TARG[0])
        spectral_DR.append(df.DR_flag[0])

photo_overlap["NIRSpec_ID"] = spectral_id
photo_overlap["z_Spec"] = spectral_z
photo_overlap["beta1300_3500"] = spectral_beta
photo_overlap["beta1300_3500_e"] = spectral_error
photo_overlap["RA_TARG"] = spectral_ra
photo_overlap["Dec_TARG"] = spectral_dec
photo_overlap["DR_flag"] = spectral_DR

photo_overlap.reset_index(drop = True, inplace = True)

# Check which filters were used when observing photometrically
weird_IDs = photo_overlap.query("F090W_KRON == 0.0 and F356W_KRON != 0.0").NIRCam_ID.tolist() # (3W7M)All except 090W,115W,150W,200W 
only_M_IDs = photo_overlap.query("F090W_KRON == 0.0 and F356W_KRON == 0.0").NIRCam_ID.tolist() # (1W2M) 182M;210M;444W
only_W_IDs = photo_overlap.query("F182M_KRON == 0.0").NIRCam_ID.tolist() # (7W2M) 7W, 335M,410M
Wand_4M_IDs = photo_overlap.query("(F430M_KRON == 0.0  or F430M_KRON != F430M_KRON) and F090W_KRON != 0.0 and F182M_KRON != 0.0").NIRCam_ID.tolist() # (7W4M), 4 M except 410,430,460M
Wand_7M_IDs = photo_overlap.query("not (F430M_KRON == 0.0  or F430M_KRON != F430M_KRON) and F090W_KRON != 0.0 and F182M_KRON != 0.0").NIRCam_ID.tolist() # (7W7M)

filterobserved = []

for index in range(len(photo_overlap)):
    if photo_overlap.NIRCam_ID[index] in only_M_IDs:
        # 1W2M
        filterobserved.append("1W2M")
    elif photo_overlap.NIRCam_ID[index] in only_W_IDs:
        # 7W2M
        filterobserved.append("7W2M")
    elif photo_overlap.NIRCam_ID[index] in Wand_4M_IDs:
        # 7W4M
        filterobserved.append("7W4M")
    elif photo_overlap.NIRCam_ID[index] in Wand_7M_IDs:
        # 7W7M
        filterobserved.append("7W7M")
    elif photo_overlap.NIRCam_ID[index] in weird_IDs:
        # 3W7M
        filterobserved.append("3W7M")
    else:
        print("")
        print(photo_overlap.NIRCam_ID[index])
        display(photo_overlap.iloc[[index]])
        
photo_overlap["filterobserved"] = filterobserved
photo_overlap.to_csv("overlap catalogue.csv", index=False)